In [47]:
import pandas as pd
import torch
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. Intro
1. 计算得到GT2(InferNet)和GT3(InferNet)对于testing set上的NLL和KL-D

# 1. Preparations
## 1.1 global settings

In [48]:
from MLP.utils import *
import os

# For GT-2
from GT_model.GT_2.SA_for_PT_funcs_delta_eq1 import *
import GT_model.GT_2.SA_for_PT_funcs_delta_eq1
# reload(GT_model.GT_2.SA_for_PT_funcs_delta_eq1)
from importlib import reload
import metric_for_GTs_func
from metric_for_GTs_func import *
reload(metric_for_GTs_func)

from MLP.Config.config_base import BaseConfig
config = BaseConfig()


## 1.2 the data path


In [49]:
ARTIFICIAL = True
noise_pct = 0.05           # 噪音占比:我们希望生成的data总体上最多浮动的百分比noise_pct
# seed = 3                 # [3,31,204,223,407]
q=1                        # q=1表示不使用quantile 计算metric
THRESHOLD = 0   # 0: 计算全部的data

# Target data
if ARTIFICIAL:
        target_path = "../../data/artificial_targets_v2_" + "noise=" + str(noise_pct)
else:
        target_path = r"../../data/targets_all"

# Output
output_path = r"../../MLP/metric_saved/"
prefix = ""
if ARTIFICIAL:
        prefix = "synthetic_"
else:
        prefix = "real_"

# Input: GT-2 params
# if ARTIFICIAL:
#         # GT-2 NN 产生的params
#         params_opitim_path_NN = r"../data/SA_PT/params_artificial_v2_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"
#         # SA 的 params
#         params_opitim_path_SA = "../data/SA_PT/params_opitim_artificial_v2_noise_"+str(noise_pct)+".csv"
# else:
#         # GT-2 NN 产生的params
#         params_opitim_path_NN_2 = r"../data/SA_PT/params_seed=" + str(seed) + ".csv"
#         params_opitim_path_NN_3 = r"../data/SA_PT/params_GT3_seed=" + str(seed) + ".csv"
#         # SA 的 params
#         params_opitim_path_SA = "../data/SA_PT/params_opitim_delta_T.csv"

# target data是target_5时 TARGET = 5
TARGET = 1

# Output path
# 3 col and 4 col
# if ARTIFICIAL:
#         NLL_metric_path_1  = r"../data/GT_metric/NLL_metric_GT_Tgt=1_e30_artificial_v_2_noise="+str(noise_pct)+".csv"
#         NLL_metric_path_2  ="../data/GT_metric/NLL_metric_GT_Tgt=1_e30_all_"+"artificial_targets_v2_" + "noise=" + str(noise_pct)+"_seed="+str(seed)+".csv"
# else:
#         NLL_metric_path_1  ="../data/GT_metric/NLL_metric_GT_Tgt=1_e30.csv"
#         # 添加的这一列和seed有关！
#         NLL_metric_path_2  ="../data/GT_metric/NLL_metric_GT_Tgt=1_e30_seed="+str(seed)+".csv"

col_names = ['GT2(NN)','GT3(NN)']

# For GT-2
alpha = -0.013581112
delta = 1
labda = 3.312402533

MIN_LOSS = 1e-30

## 1.3 Read in

In [50]:
data_key = pd.read_csv(config.data_key_path,encoding="utf-8")

# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)
len_all = len(target_all_path)
print(f"一共*{len(data_key)}*场auction")

assert len_all == len(data_key),"wrong in len_all"

一共*1276*场auction


- Random seed

In [51]:

# 简易的提取test set中的sample index
def save_data_idx_simplified(LEN):
        """
        因为objective function会被执行很多次，所以这里先保存一下idx，使得所有objective在同一组dataset上进行。
        然后在tuning时，会在shuffle_time组不同的dataset split上进行，用以取平均
        Args:
        dataset:
        opt:
        shuffle_time: The num of list of index to be generated.
        """
        shuffled_indices = np.random.permutation(LEN)

        return shuffled_indices

# 2. Compute the KL-D for GT1, GT2(InferNet), GT3(InferNet)

In [52]:
## Real
if not ARTIFICIAL:
        seed_list = [3,31,62,204,223,407,508,626]

## Synth.
else:
        seed_list = [4,31,35,66,204,407,508,512]
seed_list

[4, 31, 35, 66, 204, 407, 508, 512]

In [ ]:
GT_metric_pd_KL = pd.DataFrame()

for seed in seed_list:
        setup_seed(seed)
        shuffled_indices = save_data_idx_simplified(len_all)
        _,_,test_idx = get_data_idx(shuffled_indices, train_pct = config.train_pct, vali_pct=config.vali_pct)

        print(f"========== seed = {seed} ==========")

        if not ARTIFICIAL:
                params_opitim_path_NN_2 = r"../../data/SA_PT/params_GT2_seed=" + str(seed) + ".csv"
                params_opitim_path_NN_3 = r"../../data/SA_PT/params_GT3_seed=" + str(seed) + ".csv"
        else:
                params_opitim_path_NN_2 = r"../../data/SA_PT/params_artificial_GT2_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"
                params_opitim_path_NN_3 = r"../../data/SA_PT/params_artificial_GT3_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"

        params_opitim_NN_2 = pd.read_csv(params_opitim_path_NN_2,encoding="utf-8")
        params_opitim_NN_3 = pd.read_csv(params_opitim_path_NN_3,encoding="utf-8")

        pd_idx = 0
        metric_pd = pd.DataFrame()
        for i in test_idx:
                # Get target data
                target_path_i_path = os.path.join(target_path, target_all_path[i])
                target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
                # target = list(target_df.iloc[:,0]) # NLL 使用
                target_df_uniq = pd.DataFrame(np.unique(target_df,axis=0)) # KL-D计算必须有这一步，去掉N值重复的sample

                if target_df.shape[0] >= THRESHOLD:
                        pass
                else:   # 不计算少于THRESHOLD的auction
                        continue

                # if not ARTIFICIAL:
                #         target_df_uniq = pd.DataFrame(np.unique(target_df,axis=0)) # KL-D计算必须有这一步，去掉N值重复的sample
                # else: # 对于生成数据，不包含P列，需要手动计算
                #         target_df_group = target_df.groupby(by = ['N'],as_index=False).size()
                #         target_df_group['P'] = target_df_group['size'] / sum(target_df_group['size'])
                #
                #         target_df_uniq = pd.merge(left = target_df, right=target_df_group ,how="left", on=['N'])
                #         target_df_uniq.drop(labels='size',axis=1,inplace=True)
                #         target_df_uniq = pd.DataFrame(np.unique(target_df_uniq,axis=0)).copy()    # KL-D计算必须有这一步，去掉N值重复的sample
                #         target_df_uniq.columns = ['N','P']

                # Get the auction setting of i_th according to the test_idx
                settings_df = data_key.iloc[i,:]

                alpha_2 = params_opitim_NN_2.loc[i,'alpha'].item()
                labda_2 = params_opitim_NN_2.loc[i,'labda'].item()
                alpha_3 = params_opitim_NN_3.loc[i,'alpha'].item()

                # metric_pd.loc[pd_idx,col_names[0]] = GT1_KL(settings_df,target_df_uniq)
                metric_pd.loc[pd_idx,col_names[0]] = GT2_KL(settings_df,target_df_uniq,alpha_2,labda_2)
                metric_pd.loc[pd_idx,col_names[1]] = GT3_KL(settings_df,target_df_uniq,alpha_3)

                pd_idx += 1

        # GT_metric_pd_KL.loc[seed,col_names[0]] = np.mean(metric_pd.loc[:,col_names[0]])
        GT_metric_pd_KL.loc[seed,col_names[0]] = np.mean(metric_pd.loc[:,col_names[0]])
        GT_metric_pd_KL.loc[seed,col_names[1]] = np.mean(metric_pd.loc[:,col_names[1]])


GT_metric_pd_KL.reset_index(inplace=True)
print(GT_metric_pd_KL)

========== seed = 4 ==========
========== seed = 31 ==========
========== seed = 35 ==========
========== seed = 66 ==========


In [ ]:
GT_metric_pd_KL

In [ ]:

output_file = output_path + prefix +"InferNet_metric_KL.csv"
GT_metric_pd_KL.to_csv(output_file)
output_file

# 3. Compute the NLL for GT1, GT2(InferNet), GT3(InferNet)

In [ ]:
GT_metric_pd_NLL = pd.DataFrame()

for seed in seed_list:
        setup_seed(seed)
        shuffled_indices = save_data_idx_simplified(len_all)
        _,_,test_idx = get_data_idx(shuffled_indices, train_pct = config.train_pct, vali_pct=config.vali_pct)

        print(f"========== seed = {seed} ==========")

        if not ARTIFICIAL:
                params_opitim_path_NN_2 = r"../../data/SA_PT/params_GT2_seed=" + str(seed) + ".csv"
                params_opitim_path_NN_3 = r"../../data/SA_PT/params_GT3_seed=" + str(seed) + ".csv"
        else:
                params_opitim_path_NN_2 = r"../../data/SA_PT/params_artificial_GT2_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"
                params_opitim_path_NN_3 = r"../../data/SA_PT/params_artificial_GT3_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"

        params_opitim_NN_2 = pd.read_csv(params_opitim_path_NN_2,encoding="utf-8")
        params_opitim_NN_3 = pd.read_csv(params_opitim_path_NN_3,encoding="utf-8")

        pd_idx = 0
        metric_pd = pd.DataFrame()
        for i in test_idx:
                # Get target data
                target_path_i_path = os.path.join(target_path, target_all_path[i])
                target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
                target = list(target_df.iloc[:,0])

                if target_df.shape[0] >= THRESHOLD:
                        pass
                else:   # 不计算少于THRESHOLD的auction
                        continue

                # Get the auction setting of i_th according to the test_idx
                settings_df = data_key.iloc[i,:]

                alpha_2 = params_opitim_NN_2.loc[i,'alpha'].item()
                labda_2 = params_opitim_NN_2.loc[i,'labda'].item()
                alpha_3 = params_opitim_NN_3.loc[i,'alpha'].item()

                # Compute the NLL value of each GT models
                # metric_pd.loc[pd_idx,col_names[0]] = GT_1(settings_df,target)
                metric_pd.loc[pd_idx,col_names[0]] = GT_2_uniq(settings_df,target,alpha_2,labda_2,q=q)
                metric_pd.loc[pd_idx,col_names[1]] = GT_3_uniq(settings_df,target,alpha_3,q=q)

                pd_idx += 1

        # GT_metric_pd_NLL.loc[seed,col_names[0]] = np.mean(metric_pd.loc[:,col_names[0]])
        GT_metric_pd_NLL.loc[seed,col_names[0]] = np.mean(metric_pd.loc[:,col_names[0]])
        GT_metric_pd_NLL.loc[seed,col_names[1]] = np.mean(metric_pd.loc[:,col_names[1]])


GT_metric_pd_NLL.reset_index(inplace=True)
print(GT_metric_pd_NLL)

In [ ]:
GT_metric_pd_NLL

In [ ]:
output_file = output_path+prefix +"InferNet_metric_NLL.csv"
GT_metric_pd_NLL.to_csv(output_file)
output_file